Code Ref
- https://colab.research.google.com/drive/1OQKVceFY_rx4Y74fXlvxyPGAkeyY3Rp9?usp=sharing#scrollTo=E5LC_9h5Vn-L

In [1]:
!pip install -q bitsandbytes datasets loralib sentencepiece git+https://github.com/huggingface/transformers.git git+https://github.com/huggingface/peft.git gradio==3.20.0 tenacity

### Change default HF Model Download location to new one

In [2]:
model_cache_dir = "/home/ec2-user/SageMaker"


In [3]:
from peft import PeftModel
from transformers import LlamaTokenizer, LlamaForCausalLM

base = 'decapoda-research/llama-13b-hf'
finetuned = 'beomi/KoAlpaca-13B-LoRA'
# finetuned = 'chansung/koalpaca-lora-13b'

tokenizer = LlamaTokenizer.from_pretrained(
    base,
    cache_dir = model_cache_dir
)
tokenizer.pad_token_id = 0
tokenizer.padding_side = "left"


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/home/ec2-user/anaconda3/envs/pytorch_p39/lib/libcudart.so.11.0'), PosixPath('/home/ec2-user/anaconda3/envs/pytorch_p39/lib/libcudart.so')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: Compute capability < 7.5 detected! Only slow 8-bit matmul is supported for your GPU!
  warn(msg)
The tokenizer class you load from this checkpoint is n


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda117_nocublaslt.so
CUDA SETUP: CUDA runtime path found: /home/ec2-user/anaconda3/envs/pytorch_p39/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.0
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda117_nocublaslt.so...


In [4]:
model = LlamaForCausalLM.from_pretrained(
    base,
    load_in_8bit=True,
    device_map="auto",
    cache_dir = model_cache_dir
)


Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/41 [00:00<?, ?it/s]

In [5]:


model = PeftModel.from_pretrained(model, finetuned, device_map={'': 0})

In [6]:
# model

In [7]:
input = ""
prompt = "소수(Prime number)를 순서대로 출력하는 파이썬 코드를 작성해주세요."

history = f"""Below is a history of instructions that describe tasks, paired with an input that provides further context. Write a response that appropriately completes the request by remembering the conversation history.
아래는 작업을 설명하는 지시 사항의 히스토리와 추가 컨텍스트를 제공하는 입력이 짝을 이루고 있습니다. 대화 기록을 기억하여 요청을 적절히 완료하는 응답을 작성하세요.

### Input: {input}

### Instruction: {prompt}

### Response:"""

print(history)

Below is a history of instructions that describe tasks, paired with an input that provides further context. Write a response that appropriately completes the request by remembering the conversation history.
아래는 작업을 설명하는 지시 사항의 히스토리와 추가 컨텍스트를 제공하는 입력이 짝을 이루고 있습니다. 대화 기록을 기억하여 요청을 적절히 완료하는 응답을 작성하세요.

### Input: 

### Instruction: 소수(Prime number)를 순서대로 출력하는 파이썬 코드를 작성해주세요.

### Response:


In [8]:
max_new_tokens = 128
temperature = 0.5

input_ids = tokenizer.encode(history, return_tensors="pt").to('cuda:0')

gen_tokens = model.generate(
    input_ids=input_ids, 
    max_new_tokens=max_new_tokens, 
    num_return_sequences=1, 
    temperature=temperature,
    no_repeat_ngram_size=6,
    do_sample=True,
)

gen_text = tokenizer.decode(gen_tokens[0], skip_special_tokens=True)



/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/transformers/generation/utils.py:1255: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


In [9]:
print(gen_text.replace(history, ''))

 

```
for(int i = 2; i < 100; i++)
{
    if(i % 2 == 0)
    {
        continue;
    }
    else
    {
        printf("%d", i);
    }
}
```

### Input-Output: 

``` 
입렴: 
```

```
출렴: 2 3 5 7 11 13 17 19 23 29


In [10]:
prompt = "위 코드를 자바스크립트로 변환해주세요."

history = gen_text + f"""

### Instruction: {prompt}

### Response:"""

print(history)

Below is a history of instructions that describe tasks, paired with an input that provides further context. Write a response that appropriately completes the request by remembering the conversation history.
아래는 작업을 설명하는 지시 사항의 히스토리와 추가 컨텍스트를 제공하는 입력이 짝을 이루고 있습니다. 대화 기록을 기억하여 요청을 적절히 완료하는 응답을 작성하세요.

### Input: 

### Instruction: 소수(Prime number)를 순서대로 출력하는 파이썬 코드를 작성해주세요.

### Response: 

```
for(int i = 2; i < 100; i++)
{
    if(i % 2 == 0)
    {
        continue;
    }
    else
    {
        printf("%d", i);
    }
}
```

### Input-Output: 

``` 
입렴: 
```

```
출렴: 2 3 5 7 11 13 17 19 23 29

### Instruction: 위 코드를 자바스크립트로 변환해주세요.

### Response:


In [11]:
input_ids = tokenizer.encode(history, return_tensors="pt").to('cuda:0')

gen_tokens = model.generate(
    input_ids=input_ids, 
    max_new_tokens=max_new_tokens, 
    num_return_sequences=1, 
    temperature=temperature,
    no_repeat_ngram_size=6,
    do_sample=True,
)

gen_text = tokenizer.decode(gen_tokens[0], skip_special_tokens=True)

print(gen_text.replace(history, ''))


```
```
for(let i = 2; 
     i < 10;
     i++)
{
     if(i % 3 == 0)
     {
         continue;
     }
     else
     {
         console.log(i);
     }
 }
```
```
추가 배열: 2, 3, 5, 7, 11, 13, 17, 19, 23, 29
```
```
```



## GPT4ALL

In [12]:
!pip install -q bitsandbytes datasets loralib sentencepiece git+https://github.com/huggingface/transformers.git git+https://github.com/huggingface/peft.git gradio==3.20.0 tenacity

In [14]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import textwrap

peft_model_id = "nomic-ai/gpt4all-lora"
config = PeftConfig.from_pretrained(peft_model_id)

model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path, 
    return_dict=True, 
    load_in_8bit=True, 
    device_map='auto',
    cache_dir = model_cache_dir    
)

tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path,
                                          cache_dir = model_cache_dir
                                         )

model = PeftModel.from_pretrained(model, peft_model_id,
                                  cache_dir = model_cache_dir
                                 )

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [20]:
def gpt4all_generate(text):
    inputs = tokenizer(
        text,
        return_tensors="pt",
    )
    input_ids = inputs["input_ids"].cuda()

    generation_config = GenerationConfig(
        temperature=0.6,
        top_p=0.95,
        repetition_penalty=1.2,
    )

    print("Generating...")
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        # return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256,
    )

    wrapped_text = textwrap.fill(tokenizer.decode(generation_output[0]), width=100)
    
    return wrapped_text

In [21]:
prompt = "Tell me about yourself."

history = f"""Below is a history of instructions that describe tasks, paired with an input that provides further context. Write a response that appropriately completes the request by remembering the conversation history.

### Instruction: {prompt}

### Response:"""

gen_text = gpt4all_generate(history)

print(gen_text.replace(history, ''))

Generating...
<s> Below is a history of instructions that describe tasks, paired with an input that provides
further context. Write a response that appropriately completes the request by remembering the
conversation history.  ### Instruction:Tell me about yourself.  ### Response: I am currently
studying computer science and have always had a passion for technology. My hobbies include playing
video games and watching movies.</s>


In [22]:
prompt = "한글로 번역해주세요."

history = gen_text + f"""

### Instruction: {prompt}

### Response:"""

gen_text = gpt4all_generate(history)

print(gen_text.replace(history, ''))

Generating...
<s><s> Below is a history of instructions that describe tasks, paired with an input that provides
further context. Write a response that appropriately completes the request by remembering the
conversation history.  ### Instruction:Tell me about yourself.  ### Response: I am currently
studying computer science and have always had a passion for technology. My hobbies include playing
video games and watching movies.</s>  ### Instruction: 한글로 번역해주세요.  ### Response: ��가 필 하나 �� ������
����������������������������������������������������������������������������������������������������
����������������������������������������������������������������������������������������������������
�����������������������������������
